In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import Nucleation_Dynamics as nd
from extensisq import BS5, SSV2stab, Pr9, CK5, CFMR7osc, Pr7
import scienceplots

plt.style.use(['science', 'ieee'])

In [2]:
# Ejemplo de uso de la clase
params = {
    'temperature': 750,
    'activation_energy': 59920.2,
    'diffusivity_factor': 2e-9,
    'jump_distance': 4.6e-10,
    'molar_mass': 150.05,
    'mass_density': 2.5,
    'melting_point': 1300,
    'heat_fusion': 52 * 1e3,
    'supersaturation_ratio': 20.5,
    'sigma': 0.15, 
    'method': 'melting'
}

cluster_physics = nd.ClusterPhysics(params)

print(f"Critical radius: {cluster_physics.critical_radius}")
print(f"Critical Gibbs free energy: {cluster_physics.critical_energy_barrier}")
print(f"Critical number of atoms: {cluster_physics.critical_number_of_molecules}")

Critical radius: 8.184545454545456e-10 meter
Critical Gibbs free energy: 4.2089037887339014e-19 joule
Critical number of atoms: 23.04237422754489 dimensionless


In [3]:
MAX_NUMBER_MOLECULES = int(1e3)
number_clusters_start = 2
dt = 1e-8/cluster_physics.unbiased_jump_rate.magnitude
dt = dt*1e6

In [4]:
sim = nd.ScipyClusterDynamics(params,int(1e4),dt, number_clusters_start, MAX_NUMBER_MOLECULES)
sim.simulate(method="BDF", t_eval=None, rtol=1e-9, atol=1e-9)

y = sim.cluster_array[:,-1]


In [5]:
from extensisq.sommeijer import maxm, NFS, nfesig
from time import perf_counter

# now solve for a range of tolerances:
tols = np.logspace(-4, -7, 4)

# Lista de métodos de integración para comparar
methods = [
    ("BDF", "BDF"), 
    ("RK45", "RK45"), 
    ("Radau", "Radau"), 
    (Pr7, "Pr7"), 
    (Pr9, "Pr9"), 
    (CK5, "CK5"), 
    (CFMR7osc, "CFMR7osc"), 
    (BS5, "BS5")
]

# Ahora resuelve para un rango de tolerancias:
tols = np.logspace(-4, -7, 2)

print('{:<6} {:<8} {:<5} {:<7} {:<7} {:<7} {:<8} {:<4} {:<5}'.format('Method', 'Error', 'Tol', 'Steps', 'f-evals', 'Average', 'f-sigma', 'CPU', 's-max'))
for method, method_name in methods:
    for tol in tols:
        timer = perf_counter()
        sim = nd.ScipyClusterDynamics(params, int(1e4), dt, number_clusters_start, MAX_NUMBER_MOLECULES)
        sim.simulate(method=method, t_eval=None, rtol=tol, atol=tol)
        cpu = perf_counter() - timer
        err = np.abs(sim.cluster_array[:, -1] - y).max()
        nfs = NFS[()]
        nsteps = sim.time.size - 1 + nfs
        nfev = sim.nfev
        avg = nfev / nsteps
        print(f'{method_name:<6} {err:.4e} {tol:.0e} {nsteps:>6} ({NFS[()]}) {nfev:>7} {avg:>7.1f} {nfesig:>8} {cpu:>4.1f} {maxm[()]:>5}')

Method Error    Tol   Steps   f-evals Average f-sigma  CPU  s-max
BDF    6.6474e+01 1e-04   9999 (0)     629     0.1        0  5.3     0
BDF    7.8125e-03 1e-07   9999 (0)    1896     0.2        0 21.2     0
RK45   4.8000e+01 1e-04   9999 (0)  530342    53.0        0 14.4     0
RK45   4.8000e+01 1e-07   9999 (0)  540566    54.1        0 15.4     0
Radau  1.8282e+02 1e-04   9999 (0)    1442     0.1        0 13.1     0
Radau  8.1493e-03 1e-07   9999 (0)    7074     0.7        0 18.3     0


/home/alexandre_sureda/miniconda3/envs/neutronics/lib/python3.11/site-packages/extensisq/common.py:431: UserWarning: Your problem has a complex pair of dominant roots near the imaginary axis.  There are many recently failed steps.  You should probably change to a code intended for oscillatory problems.
  warn('Your problem has a complex pair of dominant '
/home/alexandre_sureda/miniconda3/envs/neutronics/lib/python3.11/site-packages/extensisq/common.py:456: UserWarning: Your problem has a complex pair of dominant roots and is diagnosed as stiff.  You should probably change to a code intended for stiff problems.
  warn('Your problem has a complex pair of dominant roots '
/home/alexandre_sureda/miniconda3/envs/neutronics/lib/python3.11/site-packages/extensisq/common.py:452: UserWarning: Your problem has a real dominant root and is diagnosed as stiff.  You should probably change to a code intended for stiff problems.
  warn('Your problem has a real dominant root '


Pr7    3.2000e+01 1e-04  16745 (6746)  694368    41.5        0 56.1     0
Pr7    3.2000e+01 1e-07  16960 (6961)  708470    41.8        0 69.1     0
Pr9    1.6000e+01 1e-04  27101 (17102) 1249047    46.1        0 102.2     0
Pr9    1.6000e+01 1e-07  27592 (17593) 1277130    46.3        0 127.2     0
CK5    4.0000e+01 1e-04  12396 (2397)  433857    35.0        0 12.2     0
CK5    4.0000e+01 1e-07  13074 (3075)  445777    34.1        0 13.1     0
CFMR7osc 2.4000e+01 1e-04  14866 (4867)  540518    36.4        0 15.6     0
CFMR7osc 2.4000e+01 1e-07  15645 (5646)  556514    35.6        0 16.0     0
BS5    2.5064e+01 1e-04  49842 (39843)  683649    13.7        0 19.7     0
BS5    2.4000e+01 1e-07  45795 (35796)  676593    14.8        0 19.4     0
